In [1]:
# ============================================================================
# RECOMENDADOR DE GRASAS INDUSTRIALES
# Basado en el enfoque del recomendador de películas de MovieLens
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Para reproducibilidad
np.random.seed(42)

import warnings
warnings.simplefilter('ignore')

print("=" * 70)
print("SISTEMA DE RECOMENDACIÓN DE GRASAS INDUSTRIALES")
print("=" * 70)

SISTEMA DE RECOMENDACIÓN DE GRASAS INDUSTRIALES


In [2]:
# ============================================================================
# 1. CARGA DE DATOS
# ============================================================================

# Cargar dataset principal
df = pd.read_csv("../datos/datos_grasas_Tec_limpio.csv")

print(f"\n📊 Dataset cargado: {df.shape[0]} grasas × {df.shape[1]} columnas")
print("\n🔍 Primeras 3 filas:")
df.head(3)


📊 Dataset cargado: 51 grasas × 26 columnas

🔍 Primeras 3 filas:


,idDatosGrasas,codigoGrasa,Aceite Base,Espesante,Grado NLGI Consistencia,Viscosidad del Aceite Base a 40°C. cSt,"Penetración de Cono a 25°C, 0.1mm","Punto de Gota, °C","Estabilidad Mecánica, %","Punto de Soldadura Cuatro Bolas, kgf",...,Factor de Velocidad,"Temperatura de Servicio °C, min","Temperatura de Servicio °C, max",categoria,subtitulo,descripcion,beneficios,aplicaciones,color,textura
0,1,Grasa_1,Semi-Sintetico,Complejo Sulfonato de Calcio,2.0,680.0,279.0,304,1.07,500.0,...,200000.0,5,220,Grasa,Grasa de servicio pesado para alta resistencia...,El producto es grasa lubricante de servicio pe...,Excelentes caracterÃ­sticas de resistencia al ...,Debido a su composiciÃ³n Ã³ptima de aceites de...,CafÃ©,Suave
1,2,Grasa_2,Mineral HT,Complejo Sulfonato de Calcio,1.5,460.0,300.0,304,2.60,900.0,...,300000.0,-30,150,Grasa,Grasa para lubricaciÃ³n de equipo pesado.,El producto es una grasa lubricante de gran ad...,Extremo soporte de carga.\r\n@Alto contenido d...,LubricaciÃ³n centralizada de equipo pesado.\r\...,Negro,Adherente
2,3,Grasa_3,Mineral HT,Complejo Sulfonato de Calcio,2.0,460.0,280.0,300,10.00,500.0,...,NaN,-10,150,Grasa,Grasa para lubricaciÃ³n de equipo pesado.,El producto es una grasa lubricante de gran ad...,Extremo soporte de carga.\r\n@Alto contenido d...,LubricaciÃ³n centralizada de equipo pesado.\r\...,Negro,Adherente


In [4]:
# ============================================================================
# 2. EXPLORACIÓN DE ESTRUCTURA
# ============================================================================

print("\n" + "="*70)
print("📋 INFORMACIÓN DEL DATASET")
print("="*70)

# Información general
df.info()


📋 INFORMACIÓN DEL DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 26 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   idDatosGrasas                             51 non-null     int64  
 1   codigoGrasa                               51 non-null     object 
 2   Aceite Base                               47 non-null     object 
 3   Espesante                                 45 non-null     object 
 4   Grado NLGI Consistencia                   51 non-null     float64
 5   Viscosidad del Aceite Base a 40°C. cSt    50 non-null     float64
 6   Penetración de Cono a 25°C, 0.1mm         51 non-null     float64
 7   Punto de Gota, °C                         51 non-null     int64  
 8   Estabilidad Mecánica, %                   48 non-null     float64
 9   Punto de Soldadura Cuatro Bolas, kgf      50 non-null     float64
 10  Desgaste Cuat

In [5]:
# ============================================================================
# 3. ANÁLISIS DE VALORES FALTANTES
# ============================================================================

print("\n" + "="*70)
print("🔍 VALORES FALTANTES POR COLUMNA")
print("="*70)

missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Columna': missing.index,
    'Valores Faltantes': missing.values,
    'Porcentaje (%)': missing_pct.values
})

# Mostrar solo columnas con valores faltantes
missing_df = missing_df[missing_df['Valores Faltantes'] > 0].sort_values('Valores Faltantes', ascending=False)

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("✅ No hay valores faltantes")


🔍 VALORES FALTANTES POR COLUMNA
                                 Columna  Valores Faltantes  Porcentaje (%)
                            Registro NSF                 45       88.235294
                Indice de Carga-Desgaste                 41       80.392157
                      Corrosión al Cobre                 35       68.627451
               Desgaste Cuatro Bolas, mm                 24       47.058824
                     Factor de Velocidad                 15       29.411765
Resistencia al Lavado por Agua a 80°C, %                  7       13.725490
                               Espesante                  6       11.764706
                     Carga Timken Ok, lb                  5        9.803922
                             Aceite Base                  4        7.843137
                 Estabilidad Mecánica, %                  3        5.882353
  Viscosidad del Aceite Base a 40°C. cSt                  1        1.960784
    Punto de Soldadura Cuatro Bolas, kgf               

In [6]:
# ============================================================================
# 4. ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS
# ============================================================================

print("\n" + "="*70)
print("📊 ESTADÍSTICAS DE VARIABLES TÉCNICAS CLAVE")
print("="*70)

# Seleccionar columnas técnicas principales
tech_cols = [
    'Grado NLGI Consistencia',
    'Viscosidad del Aceite Base a 40°C. cSt',
    'Penetración de Cono a 25°C, 0.1mm',
    'Punto de Gota, °C',
    'Estabilidad Mecánica, %',
    'Punto de Soldadura Cuatro Bolas, kgf',
    'Temperatura de Servicio °C, min',
    'Temperatura de Servicio °C, max'
]

df[tech_cols].describe().round(2)


📊 ESTADÍSTICAS DE VARIABLES TÉCNICAS CLAVE


,Grado NLGI Consistencia,Viscosidad del Aceite Base a 40°C. cSt,"Penetración de Cono a 25°C, 0.1mm","Punto de Gota, °C","Estabilidad Mecánica, %","Punto de Soldadura Cuatro Bolas, kgf","Temperatura de Servicio °C, min","Temperatura de Servicio °C, max"
count,51.00,50.00,51.00,51.00,48.00,50.00,51.00,51.00
mean,1.52,579.60,300.19,458.12,5.39,486.20,-10.35,153.53
std,0.83,734.16,44.23,1363.37,3.61,221.48,8.94,31.10
min,0.00,40.00,230.00,110.00,0.80,250.00,-30.00,90.00
25%,1.00,120.00,275.00,252.50,2.48,315.00,-15.00,150.00
50%,1.50,430.00,295.00,280.00,5.00,500.00,-10.00,150.00
75%,2.00,680.00,320.00,304.00,6.88,620.00,-5.00,160.00
max,3.00,3800.00,460.00,9999.00,14.00,900.00,5.00,240.00


In [7]:
print(df[tech_cols].describe().round(2))

       Grado NLGI Consistencia  Viscosidad del Aceite Base a 40°C. cSt  \
count                    51.00                                   50.00   
mean                      1.52                                  579.60   
std                       0.83                                  734.16   
min                       0.00                                   40.00   
25%                       1.00                                  120.00   
50%                       1.50                                  430.00   
75%                       2.00                                  680.00   
max                       3.00                                 3800.00   

       Penetración de Cono a 25°C, 0.1mm  Punto de Gota, °C  \
count                              51.00              51.00   
mean                              300.19             458.12   
std                                44.23            1363.37   
min                               230.00             110.00   
25%               